# lDE20 Image Processing Notebook - Run 1

See './Image_Analysis/Examples/1_Core_Image_Processing.ipynb' for a documented example of notebook usage.

#### Imports

In [ ]:
import trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
# addition of active memory manager
import dask
dask.config.set({'distributed.scheduler.active-memory-manager.start': True});
dask.config.set({'distributed.scheduler.worker-ttl': "5m"});
dask.config.set({'distributed.scheduler.allowed-failures': 100});

dask_wd = "/home/de64/scratch/de64/dask"

In [ ]:
### Dark Image

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/20X_Dark.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/20X_Dark.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

In [ ]:
### MCherry

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/mCherry_20x_Phase.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/mCherry_20x_Phase.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

In [ ]:
import tifffile
from matplotlib import pyplot as plt

In [ ]:
dark_outputpath = "/home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/20X_Dark.tiff"
Mcherry_outputpath = "/home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/mCherry_20x_Phase.tiff"

fflist = [dark_outputpath,Mcherry_outputpath]

fig, axs = plt.subplots(figsize=(5, 6), nrows= 1, ncols = 2)

for i in range(2):
    axs[i].imshow(tifffile.imread(fflist[i]))
    axs[i].set_title(fflist[i].split('/')[-1].split(".")[0])
plt.tight_layout()

# Part 1: Growth/Division

In [ ]:
# headpath = "/home/de64/scratch/de64/sync_folder/2024-10-25_DE828_AHL_Titration/"
# nd2file = "/home/de64/scratch/de64/sync_folder/2024-10-25_DE828_AHL_Titration/Experiment.nd2"

headpath = "/home/de64/scratch/de64/2024-11-08_DE828_AHL_Titration_2/"
nd2file = "/home/de64/scratch/de64/2024-11-08_DE828_AHL_Titration_2/Experiment.nd2"

# headpath = "/home/de64/scratch/de64/sync_folder/2024-10-29_Translation_Series/"
# nd2file = "/home/de64/scratch/de64/sync_folder/2024-10-29_Translation_Series/Experiment.nd2"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=100,
    n_workers_min=50,
    memory="4GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

##### Perform Extraction

In [ ]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    nd2file,
    headpath,
    tpts_per_file=50,
    ignore_fovmetadata=False,
    nd2reader_override={"z_levels": [], "z_coordinates": []},
)

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

#### List of paths to flat fields
 - /home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/mCherry_20x_Phase.tiff
 - /home/de64/scratch/de64/sync_folder/2024-10-27_Flat_Fields_Iris_9_20x/20X_Dark.tiff

In [ ]:
hdf5_extractor.inter_set_flatfieldpaths()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

## Kymographs

### Test Parameters



##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, paramfile=True
)

##### Begin Kymograph Cropping 

In [ ]:
kymoclust.generate_kymographs(dask_controller)

In [ ]:
lane_overlay_handle = tr.lane_overlay(headpath,persist_data=True)
overlay = lane_overlay_handle.view_overlay(size=2500,vmin=0,vmax=4000)

In [ ]:
overlay

In [ ]:
lane_overlay_handle.save_rows()

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller,trench_timepoints_per_file=10000)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

## Fluorescence Segmentation

In [ ]:
import pickle as pkl

with open("/home/de64/scratch/de64/sync_folder/2023-01-14_lDE20_Run_9/Growth_Division/fluorescent_segmentation.par","rb") as infile:
    test = pkl.load(infile)

print(test)

### Test Parameters

##### Initialize the interactive segmentation class

As a first step, initialize the `tr.fluo_segmentation_interactive` class that will be handling all steps of generating a segmentation. 

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
interactive_segmentation.choose_seg_channel_inter()

#### Import data

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

#### Shutdown Dask

In [ ]:
dask_controller.shutdown()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=150,
    n_workers_min=20,
    memory="1GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath, paramfile=True)

In [ ]:
segment.dask_segment(dask_controller)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Lineage Tracing

### Test Parameters

In [ ]:
score_function = tr.tracking.scorefn(
    headpath,
    "fluorsegmentation",
    size_attr='axis_major_length',
    u_size=0.08,
    sig_size=0.03,
    u_pos=0.1,
    sig_pos=0.05,
    w_merge=0.,
)

In [ ]:
score_function.interactive_scorefn()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, "fluorsegmentation", ScoreFn=score_function, edge_limit=2,
)
data, orientation = score_function.output.result

In [ ]:
Tracking_Solver.interactive_tracking(data, orientation)

In [ ]:
Tracking_Solver.save_params()

### Generate Lineage Traces

In [ ]:
# dask_controller = tr.trcluster.dask_controller(
#     walltime="1:00:00",
#     local=False,
#     n_workers=300,
#     n_workers_min=20,
#     memory="1GB",
#     death_timeout=60.,
#     working_directory=dask_wd,
# )
# dask_controller.startdask()
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=50,
    n_workers_min=50,
    memory="8GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, 
    "fluorsegmentation", 
    paramfile=True, 
    size_estimation=True,
    size_estimation_method='Perimeter/Area',
    props_list=['area'], 
    props_to_unpack={},
    pixel_scaling_factors={'area': 2},
    intensity_props_list=['mean_intensity'] 
)

In [ ]:
Tracking_Solver.compute_all_lineages(dask_controller,entries_per_partition = 30000, overwrite=True)

#### Crop sgRNA Experiment / AHL Experiment 2 to Center Square
- Focus was off for these two experiments in the periphery

In [ ]:
import numpy as np
import scipy as sp
import dask.dataframe as dd
import pandas as pd

from matplotlib import pyplot as plt

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=40,
    n_workers_min=20,
    memory="4GB",
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
lineage_df = dd.read_parquet("/home/de64/scratch/de64/sync_folder/2024-11-08_DE828_AHL_Titration_2/lineage",calculate_divisions=True)

lineage_df_sample = lineage_df.sample(frac=0.01).compute()

In [ ]:
fig = plt.figure(figsize=(5,5))

y_lower,y_upper = (110,500)
x_lower,x_upper = (100,500)

plt.gca().invert_yaxis()

plt.scatter(lineage_df_sample['Cell X (local)'],lineage_df_sample['Cell Y (local)'],s=1)

In [ ]:
lineage_df_mask = (lineage_df['Cell X (local)']>x_lower)&(lineage_df['Cell X (local)']<x_upper)&\
                (lineage_df['Cell Y (local)']>y_lower)&(lineage_df['Cell Y (local)']<y_upper)

lineage_df_pos_filtered = lineage_df[lineage_df_mask]

filtered_trenchid_list = list(lineage_df_pos_filtered["trenchid"].unique().compute())

In [ ]:
filtered_trenchid_list = list(lineage_df_pos_filtered["trenchid"].unique().compute())

In [ ]:
lineage_df_trenchid_mask = lineage_df["trenchid"].apply(lambda x: x in filtered_trenchid_list)

lineage_df_trench_filtered = lineage_df[lineage_df_trenchid_mask]

In [ ]:
dd.to_parquet(lineage_df_trench_filtered, "/home/de64/scratch/de64/sync_folder/2024-11-08_DE828_AHL_Titration_2/lineage_crop",engine='pyarrow',compression='gzip',write_metadata_file=True,overwrite=True)

In [ ]:
dask_controller.shutdown()